In [1]:
import pandas as pd
import glob, os

In [2]:
path = r'..\999_동영상 연습예제\8장\OpinosisDataset1.0\topics'

In [5]:
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list = []
opinion_text = []

for file_ in all_files:
    
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
    
    filename_ = file_.split('\\')[-1]
    filename = filename_.split('.')[0]
    
    filename_list.append(filename)
    opinion_text.append(df.to_string())

C:\Users\sejin\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  import sys


In [20]:
df.values[0:2]

array([["I also thought the the voice prompts of the 750 where more pleasant sounding than the 255w's ."],
       [' The 750 sounds more like a real female voice, where the 255w sounds more robotic .']],
      dtype=object)

In [7]:
filename_

'voice_garmin_nuvi_255W_gps.txt.data'

In [8]:
filename

'voice_garmin_nuvi_255W_gps'

In [18]:
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,", and is very, very acc..."
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and..."
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my ...
3,battery-life_ipod_nano_8gb,short battery life I moved up from a...
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 2..."


In [45]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

In [55]:
remove_punct_dict = {}
for punct in string.punctuation:
    print(punct)
    remove_punct_dict = dict(ord(punct), None )

print(remove_punct_dict)

!


TypeError: dict expected at most 1 arguments, got 2

In [64]:
remove_punct_dict = dict((ord(punct),None) for punct in string.punctuation)
remove_punct_dict

{33: None,
 34: None,
 35: None,
 36: None,
 37: None,
 38: None,
 39: None,
 40: None,
 41: None,
 42: None,
 43: None,
 44: None,
 45: None,
 46: None,
 47: None,
 58: None,
 59: None,
 60: None,
 61: None,
 62: None,
 63: None,
 64: None,
 91: None,
 92: None,
 93: None,
 94: None,
 95: None,
 96: None,
 123: None,
 124: None,
 125: None,
 126: None}

In [36]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [34]:
ord('!')

33

In [71]:
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_puct_dict)))

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                             ngram_range=(1,2), min_df=0.05, max_df=0.85)

feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

C:\Users\sejin\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [78]:
print(LemNormalize('faTher'))
print(LemNormalize('faTher$$'))
print(LemNormalize('faTher mother'))

['father']
['father']
['father', 'mother']


In [83]:
feature_vect

<51x2409 sparse matrix of type '<class 'numpy.float64'>'
	with 16507 stored elements in Compressed Sparse Row format>

In [85]:
pd.DataFrame(feature_vect).head()

,0
0,"(0, 404)\t0.7912382586976787\n (0, 1014)\t0..."
1,"(0, 1758)\t0.03946345085374527\n (0, 758)\t..."
2,"(0, 818)\t0.02713769634953538\n (0, 1315)\t..."
3,"(0, 404)\t0.017135894827923288\n (0, 818)\t..."
4,"(0, 404)\t0.0055037702413902635\n (0, 818)\..."


In [86]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=5, max_iter=1000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [87]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,", and is very, very acc...",3
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and...",0
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my ...,4
3,battery-life_ipod_nano_8gb,short battery life I moved up from a...,4
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 2...",4


In [92]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and...",0
32,room_holiday_inn_london,"We arrived at 23,30 hours and they could n...",0
30,rooms_bestwestern_hotel_sfo,"Great Location , Nice Rooms , Helpless...",0
31,rooms_swissotel_chicago,The Swissotel is one of our favorite hotel...,0


In [93]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
13,food_holiday_inn_london,The room was packed to capacity with queu...,1
14,food_swissotel_chicago,The food for our event was deli...,1
15,free_bestwestern_hotel_sfo,The wine reception is a great idea as it i...,1
20,location_bestwestern_hotel_sfo,"Good Value good location , ideal ...",1
21,location_holiday_inn_london,Great location for tube and we crammed in...,1
24,parking_bestwestern_hotel_sfo,Parking was expensive but I think this is ...,1
28,price_holiday_inn_london,"All in all, a normal chain hotel on a nice...",1
38,service_bestwestern_hotel_sfo,Both of us having worked in tourism for o...,1
39,service_holiday_inn_london,"not customer, oriented hotelvery low servi...",1
40,service_swissotel_hotel_chicago,Mediocre room and service for a very extr...,1


In [95]:
from sklearn.cluster import KMeans

km_cluster_3 = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster_3.fit(feature_vect)
cluster_label_3 = km_cluster_3.labels_
cluster_centers_3 = km_cluster_3.cluster_centers_

document_df['cluster_label_3'] = cluster_label_3

In [96]:
document_df.head()

,filename,opinion_text,cluster_label,cluster_label_3
0,accuracy_garmin_nuvi_255W_gps,", and is very, very acc...",3,0
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and...",0,1
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my ...,4,0
3,battery-life_ipod_nano_8gb,short battery life I moved up from a...,4,0
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 2...",4,0


In [101]:
document_df.sort_values(by='cluster_label_3', ascending=False)

,filename,opinion_text,cluster_label,cluster_label_3
25,performance_honda_accord_2008,"Very happy with my 08 Accord, performance i...",2,2
7,comfort_toyota_camry_2007,Ride seems comfortable and gas mileage fa...,2,2
29,quality_toyota_camry_2007,I previously owned a Toyota 4Runner which ...,2,2
37,seats_honda_accord_2008,Front seats are very uncomfor...,2,2
22,mileage_honda_accord_2008,"It's quiet, get good gas mileage and look...",2,2
18,interior_toyota_camry_2007,"First of all, the interior has way too ma...",2,2
16,gas_mileage_toyota_camry_2007,Ride seems comfortable and gas mileage fa...,2,2
17,interior_honda_accord_2008,I love the new body style and the interior...,2,2
6,comfort_honda_accord_2008,"Drivers seat not comfortable, the car its...",2,2
45,staff_bestwestern_hotel_sfo,Staff are friendly and hel...,1,1


In [102]:
feature_vect.shape

(51, 2409)

In [103]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape:', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape: (5, 2409)
[[0.         0.         0.00676386 ... 0.         0.         0.        ]
 [0.         0.00227114 0.00115031 ... 0.00434427 0.00465884 0.        ]
 [0.         0.00137309 0.         ... 0.         0.         0.        ]
 [0.01580558 0.         0.         ... 0.         0.         0.        ]
 [0.02040764 0.         0.         ... 0.         0.         0.0090591 ]]


In [106]:
cluster_centers_3 = km_cluster_3.cluster_centers_
print('cluster_centers shape:', cluster_centers_3.shape)
print(cluster_centers_3)

cluster_centers shape: (3, 2409)
[[0.0174987  0.         0.         ... 0.         0.         0.00452955]
 [0.         0.00170335 0.0025537  ... 0.0032582  0.00349413 0.        ]
 [0.         0.00152566 0.         ... 0.         0.         0.        ]]


In [119]:
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[::-1]
    
    for cluster_num in range(clusters_num):
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_feautres_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details

In [120]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reviews 파일명:',cluster_detail['filenames'][:7])
        print('============================================\n')

In [121]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model = km_cluster, cluster_data=document_df,
                                     feature_names=feature_names, clusters_num=3, top_n_features=10)
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['ideal', 'guy', 'guest', 'ground floor', 'ground', 'grocery', 'greeted', 'great wa', 'great view', 'guy wa']
Reviews 파일명: ['bathroom_bestwestern_hotel_sfo', 'rooms_bestwestern_hotel_sfo', 'rooms_swissotel_chicago', 'room_holiday_inn_london']

####### Cluster 1
Top features: ['ideal', 'long distance', 'long battery', 'london', 'location wonderful', 'location wa', 'location thought', 'location stay', 'location staff', 'location service']
Reviews 파일명: ['food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwestern_hotel_sfo', 'location_bestwestern_hotel_sfo', 'location_holiday_inn_london', 'parking_bestwestern_hotel_sfo', 'price_holiday_inn_london']

####### Cluster 2
Top features: ['0 5', 'luggage', 'lovely hotel', 'lovely', 'love small', 'love pedometer', 'lov', 'lounge', 'lot great', 'lot easier']
Reviews 파일명: ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_honda_accord_2008', 'interior_toyot

In [125]:
cluster_details

{0: {'cluster': 0,
  'top_features': ['ideal',
   'guy',
   'guest',
   'ground floor',
   'ground',
   'grocery',
   'greeted',
   'great wa',
   'great view',
   'guy wa'],
  'top_feautres_value': [0.0035045528576554646,
   0.0,
   0.012746177359552385,
   0.0023182137946106216,
   0.002054548111707642,
   0.0,
   0.0009429914039322694,
   0.0,
   0.008851594499925421,
   0.0],
  'filenames': ['bathroom_bestwestern_hotel_sfo',
   'rooms_bestwestern_hotel_sfo',
   'rooms_swissotel_chicago',
   'room_holiday_inn_london']},
 1: {'cluster': 1,
  'top_features': ['ideal',
   'long distance',
   'long battery',
   'london',
   'location wonderful',
   'location wa',
   'location thought',
   'location stay',
   'location staff',
   'location service'],
  'top_feautres_value': [0.0034420217866795756,
   0.0,
   0.0,
   0.009901900530586061,
   0.003538727602528604,
   0.020627919118389595,
   0.0026907741528848874,
   0.00367266015940954,
   0.0032013599741019354,
   0.009883415680351932],


In [123]:
cluster_details[0]

{'cluster': 0,
 'top_features': ['ideal',
  'guy',
  'guest',
  'ground floor',
  'ground',
  'grocery',
  'greeted',
  'great wa',
  'great view',
  'guy wa'],
 'top_feautres_value': [0.0035045528576554646,
  0.0,
  0.012746177359552385,
  0.0023182137946106216,
  0.002054548111707642,
  0.0,
  0.0009429914039322694,
  0.0,
  0.008851594499925421,
  0.0],
 'filenames': ['bathroom_bestwestern_hotel_sfo',
  'rooms_bestwestern_hotel_sfo',
  'rooms_swissotel_chicago',
  'room_holiday_inn_london']}

In [124]:
cluster_details[0]['cluster']

0

In [126]:
cluster_details[0]['top_features']

['ideal',
 'guy',
 'guest',
 'ground floor',
 'ground',
 'grocery',
 'greeted',
 'great wa',
 'great view',
 'guy wa']

In [129]:
centroid_feature_ordered_ind = km_cluster.cluster_centers_.argsort()[::-1]

In [138]:
centroid_feature_ordered_ind[0]

array([1204, 1105, 1104, ..., 1332, 1904,  506], dtype=int64)

In [132]:
km_cluster.cluster_centers_

array([[0.        , 0.        , 0.00676386, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00227114, 0.00115031, ..., 0.00434427, 0.00465884,
        0.        ],
       [0.        , 0.00137309, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01580558, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.02040764, 0.        , 0.        , ..., 0.        , 0.        ,
        0.0090591 ]])

In [137]:
km_cluster.cluster_centers_.sort()

In [136]:
top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]